All the methods in this notebook are based on this paper from F. Kamiran and T. Calders: [Data preprocessing techniques for classification without discrimination](https://doi.org/10.1007/s10115-011-0463-8)

### Context and objectives

Our goal will be to measure and mitigate bias on a classifier. The dataset that will be used is the statlog_german_credit_data dataset.

Please note that this dataset is quite old and hard to understand due to its complicated system of categories and symbols. As such, we turn to this simplified version of the dataset: [German Credit](https://www.kaggle.com/datasets/kabure/german-credit-data-with-risk).
Several columns were ignored because they were considered unimportant to the author, but it does not really matter for our case.  
You can still check the original [here](https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data). Its description mentions: 
> This dataset classifies people described by a set of attributes as good or bad credit risks.

As such, the objective of the classifier we are trying to build is to predict whether a client will repay the loan based on their application. The model might perform very well by leveraging all sorts of attributes. However, some of these features might be illegal to use even if they play a significant role in the predictions. For example, you would not want to use gender, ethnicity or age to make a decision (some laws actually ban such discrimination).
 
**In this notebook, we will measure the gender-related bias in a dataset and a classifier.**

### Import and prepare the data

In [1]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from giskard import Model, Dataset, SlicingFunction

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_data = pd.read_csv("german_credit_data_with_risk.csv")
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1000 non-null   int64 
 1   Age               1000 non-null   int64 
 2   Sex               1000 non-null   object
 3   Job               1000 non-null   int64 
 4   Housing           1000 non-null   object
 5   Saving accounts   817 non-null    object
 6   Checking account  606 non-null    object
 7   Credit amount     1000 non-null   int64 
 8   Duration          1000 non-null   int64 
 9   Purpose           1000 non-null   object
 10  Risk              1000 non-null   object
dtypes: int64(5), object(6)
memory usage: 86.1+ KB


We remove the first column which is just another index.

In [3]:
raw_data.drop('Unnamed: 0', axis=1, inplace=True)
raw_data.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


In [4]:
COLUMN_TYPES = {
    "Sex": "category",
    "Job": "category",
    "Age": "numeric",
    "Housing": "category",
    "Saving accounts": "category",
    "Checking account": "category",
    "Credit amount": "numeric",
    "Duration": "numeric",
    "Purpose": "category",
}

TARGET_COLUMN_NAME = "Risk"

COLUMNS_TO_SCALE = [key for key in COLUMN_TYPES.keys() if COLUMN_TYPES[key] == "numeric"]
COLUMNS_TO_ENCODE = [key for key in COLUMN_TYPES.keys() if COLUMN_TYPES[key] == "category"]

In [5]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="no")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
])

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, COLUMNS_TO_SCALE),
    ("cat", categorical_transformer, COLUMNS_TO_ENCODE),
])

In the original dataset, some features such as `Saving accounts` or `Checking accounts` could be set to a value representing *no account* or *unknown account*, which were translated as null in this dataset. Thus, we will just fill the null values with *no*.

Now there are a lot of `object`type features, for which one-hot encoding works perfectly.

For the columns of type `int64` that have large range (`Credit Amount`, `Age` and `Duration`), we scale the features to ensure that some of them do not dominate the regression model simply because of their larger ranges.

We now define the protected attribute and the values it can take, as well as the possible outcomes.

In [6]:
protected_attribute = 'Sex'
protected_value = 'male'
unprotected_value = 'female'
positive_outcome = 'good'
negative_outcome = 'bad'

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(raw_data.drop(columns=TARGET_COLUMN_NAME), raw_data[TARGET_COLUMN_NAME],
                                                    test_size=0.2, random_state=42)

train_set = pd.concat([X_train, Y_train], axis=1)

### Compute metrics to measure discrimination

Now let's check for discrimination.
We compute probabilities by selecting and counting rows. Then, we can then compute Statistical Parity Difference (SPD) and Disparate Impact (DI), which are defined as such, with $Group$ describing the group the sample belongs to (can be defavorited, noted $b$, or favorited, noted $w$) and $Class$ being its label ($+$ or $-$):
$$SPD = P(y_{pred} = + ∣ Group = b) - P(y_{pred} = + ∣ Group = w)$$
$$DI = \frac{P(y_{pred} = + ∣ Group = b)}{P(y_{pred} = + ∣ Group = w)}$$

In our case, $Group = b$ means that the applicant is female while $Group = w$ means it is a male.
These metrics directly compare the probability of a favorable outcome for a defavored group against that of a favored group.
We consider SPD to be relevant as the test statistic that would be used for a two-sample t test (assuming unknown and potentially different variances) would rely on it (see formula below). We also consider DI to show that there are other metrics that could very much be used.

$$ \frac{\bar x_1 - \bar x_2}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}} = \frac{SPD}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}, $$
with $\bar x_i$ being the proportion the positive samples from group i (for example a men and women that were given a loan), $s_i$ and $n_i$ being the empirical standard deviation and the size of group i.

As such, taking the difference in probability makes sense when measuring the severity of the discrimination at hand.
SPD should be as close to 0 as possible while DI should be closer to 1.

We will compute them by using the following:

$$ P(y_{pred} = X ∣ Group = Y) = \frac{|\{X \in D | y_{pred} = X and Group = Y\}|}{|\{X \in D | Group = Y\}|},$$

D being the dataset we use.

### Let's check for discrimination from within the dataset

To get a better understanding of how everything works beneath the surface, let's compute SPD and DI values by hand for the training set.

In [8]:
n_fav = (train_set[protected_attribute] == protected_value).sum()
n_defav = (train_set[protected_attribute] == unprotected_value).sum()

counts = train_set.groupby([protected_attribute, TARGET_COLUMN_NAME]).size()
spd = counts[unprotected_value][positive_outcome]/n_defav - counts[protected_value][positive_outcome]/n_fav
di = (counts[unprotected_value][positive_outcome]/n_defav) / (counts[protected_value][positive_outcome]/n_fav)

print(f"SPD = {spd}\nDI = {di}")

SPD = -0.08931095151567592
DI = 0.8771693210892719


We can clearly that there is bias within the dataset, as women are 9% less likely to get a loan compared to men. 

### Building a classifier

Let's build a very basic classifier that we will then evaluate using Giskard's methods

In [9]:
test_set = pd.concat([X_test, Y_test], axis=1)
giskard_dataset = Dataset(
    df=test_set,  # A pandas.DataFrame containing raw data (before pre-processing) and including ground truth variable.
    target=TARGET_COLUMN_NAME,  # Ground truth variable
    name="credit scoring", # Optional: Give a name to your dataset
    cat_columns=COLUMNS_TO_ENCODE,
    column_types=COLUMN_TYPES
)

2024-05-24 17:48:26,898 pid:8041 MainThread giskard.datasets.base INFO     Your 'pandas.DataFrame' is successfully wrapped by Giskard's 'Dataset' wrapper class.


In [10]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=800))
])

pipeline.fit(X_train, Y_train)

pred_train = pipeline.predict(X_train)
pred_test = pipeline.predict(X_test)

print(classification_report(Y_test, pred_test))

              precision    recall  f1-score   support

         bad       0.65      0.41      0.50        59
        good       0.79      0.91      0.84       141

    accuracy                           0.76       200
   macro avg       0.72      0.66      0.67       200
weighted avg       0.74      0.76      0.74       200



We wrap the model with Giskard and make sure everything is running smoothly.

In [11]:
giskard_model = Model(
    model=pipeline,
    # A prediction function that encapsulates all the data pre-processing steps and that could be executed with the dataset used by the scan.
    model_type="classification",  # Either regression, classification or text_generation.
    name="Credit scoring classifier",  # Optional.
    classification_labels=pipeline.classes_.tolist(),
    # Their order MUST be identical to the prediction_function's output order.
)

print(classification_report(Y_test, pipeline.classes_[giskard_model.predict(giskard_dataset).raw_prediction]))

2024-05-24 17:48:33,675 pid:8041 MainThread giskard.models.automodel INFO     Your 'model' is successfully wrapped by Giskard's 'SKLearnModel' wrapper class.
2024-05-24 17:48:33,685 pid:8041 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'Age': 'int64', 'Sex': 'object', 'Job': 'int64', 'Housing': 'object', 'Saving accounts': 'object', 'Checking account': 'object', 'Credit amount': 'int64', 'Duration': 'int64', 'Purpose': 'object'} to {'Age': 'int64', 'Sex': 'object', 'Job': 'int64', 'Housing': 'object', 'Saving accounts': 'object', 'Checking account': 'object', 'Credit amount': 'int64', 'Duration': 'int64', 'Purpose': 'object'}
2024-05-24 17:48:33,692 pid:8041 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (200, 10) executed in 0:00:00.016139
              precision    recall  f1-score   support

         bad       0.65      0.41      0.50        59
        good       0.79      0.91      0.84       141

    accuracy              

### Evaluating your model with Giskard

We are now ready to run the Giskard tests!

In [12]:
from giskard.testing import test_disparate_impact
from giskard.testing import test_statistical_parity_difference

protected_slicing_function=SlicingFunction(lambda df: df[df[protected_attribute] == protected_value], row_level=False)
unprotected_slicing_function=SlicingFunction(lambda df: df[df[protected_attribute] == unprotected_value], row_level=False)

result_di = test_disparate_impact(model=giskard_model, dataset=giskard_dataset, protected_slicing_function=protected_slicing_function, unprotected_slicing_function=unprotected_slicing_function, positive_outcome=positive_outcome)
result_spd = test_statistical_parity_difference(model=giskard_model, dataset=giskard_dataset, protected_slicing_function=protected_slicing_function, unprotected_slicing_function=unprotected_slicing_function, positive_outcome=positive_outcome)

In [13]:
result_di.execute()

2024-05-24 17:48:33,749 pid:8041 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'Age': 'int64', 'Sex': 'object', 'Job': 'int64', 'Housing': 'object', 'Saving accounts': 'object', 'Checking account': 'object', 'Credit amount': 'int64', 'Duration': 'int64', 'Purpose': 'object'} to {'Age': 'int64', 'Sex': 'object', 'Job': 'int64', 'Housing': 'object', 'Saving accounts': 'object', 'Checking account': 'object', 'Credit amount': 'int64', 'Duration': 'int64', 'Purpose': 'object'}
2024-05-24 17:48:33,760 pid:8041 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (144, 10) executed in 0:00:00.027997
2024-05-24 17:48:33,772 pid:8041 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'Age': 'int64', 'Sex': 'object', 'Job': 'int64', 'Housing': 'object', 'Saving accounts': 'object', 'Checking account': 'object', 'Credit amount': 'int64', 'Duration': 'int64', 'Purpose': 'object'} to {'Age': 'int64', 'Sex': 'object', 'Job': 

Test succeeded
               Metric: 1.16
                - [TestMessageLevel.INFO] min_threshold = 0.8, max_threshold = 1.25

In [14]:
result_spd.execute()

2024-05-24 17:48:33,836 pid:8041 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'Age': 'int64', 'Sex': 'object', 'Job': 'int64', 'Housing': 'object', 'Saving accounts': 'object', 'Checking account': 'object', 'Credit amount': 'int64', 'Duration': 'int64', 'Purpose': 'object'} to {'Age': 'int64', 'Sex': 'object', 'Job': 'int64', 'Housing': 'object', 'Saving accounts': 'object', 'Checking account': 'object', 'Credit amount': 'int64', 'Duration': 'int64', 'Purpose': 'object'}
2024-05-24 17:48:33,838 pid:8041 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (144, 10) executed in 0:00:00.009964
2024-05-24 17:48:33,845 pid:8041 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'Age': 'int64', 'Sex': 'object', 'Job': 'int64', 'Housing': 'object', 'Saving accounts': 'object', 'Checking account': 'object', 'Credit amount': 'int64', 'Duration': 'int64', 'Purpose': 'object'} to {'Age': 'int64', 'Sex': 'object', 'Job': 

Test failed
               Metric: 0.12
                - [TestMessageLevel.INFO] thresh = 0.1

We can conclude from these figures that there **our classifier exhibits discrimination** based on sex which is highly undesirable. Women are 12% less likely to get a loan than men. Still they pass the 80% rule of disparate impact.

---